대회 목표<br><br>
이 "시작하기" 대회에서는 시계열 예측 기능을 사용하여 에콰도르에 본사를 둔 대형 식료품 소매업체인 Corpacion Favorita의 데이터에 대한 매장 매출을 예측합니다.<br><br>
구체적으로 서로 다른 페이버리타 매장에서 판매되는 수천 개의 품목에 대한 단위 매출을 보다 정확하게 예측하는 모델을 구축하게 됩니다. 날짜, 매장 및 품목 정보, 프로모션 및 단위 판매에 대한 접근 가능한 교육 데이터 세트를 통해 머신러닝 기술을 연습할 수 있습니다.<br><br>
이 대회에서는 에콰도르에 위치한 Favorita 매장에서 판매되는 수천 개의 제품군에 대한 매출을 예측하게 됩니다. 교육 데이터에는 날짜, 매장 및 제품 정보, 해당 품목의 프로모션 여부, 판매 번호 등이 포함됩니다. 추가 파일에는 모델을 구축하는 데 유용할 수 있는 추가 정보가 포함되어 있습니다.

In [1]:
import numpy as np
import pandas as pd

In [48]:
import pandas as pd
import numpy as np
import calendar

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
import plotly.offline as offline
import plotly.graph_objs as go
offline.init_notebook_mode(connected = True)

---

# EDA

## 1. 데이터와 변수 이해

In [3]:
train.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0.0
1,1,2013-01-01,1,BABY CARE,0.0,0.0
2,2,2013-01-01,1,BEAUTY,0.0,0.0
3,3,2013-01-01,1,BEVERAGES,0.0,0.0
4,4,2013-01-01,1,BOOKS,0.0,0.0


데이터 설명<br><br>
store_nbr: 상점, 등간변수<br>
family: 판매되는 제품의 유형, 등간변수<br>
sales: 특정 가게에서 특정 날에 판매된 제품군의 총액. 연속형 변수<br>
onpromotion: 특정 가게에서 특정 날에 프로모션된 제품군 내의 아이템 개수. 연속형 변수<br><br>


In [30]:
print(f'기간: {train.date[0]} ~ {train.date.iloc[-1,]}')
print(f'가게 개수: {train.store_nbr.nunique()}')
print(f'제품군 개수: {train.family.nunique()}')

기간: 2013-01-01 ~ 2017-08-15
가게 개수: 54
제품군 개수: 33


In [39]:
stores.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


데이터 설명<br><br>
store_nbr: 상점, 등간변수<br>
city: 위치한 도시, 등간변수<br>
state: 국가. 등간변수<br>
type: 가게 유형. 등간변수<br>
cluster: 유사한 가게들의 그룹. 등간변수

In [37]:
print(f'도시 개수: {stores.city.nunique()}')
print(f'국가 개수: {stores.state.nunique()}')
print(f'타입 개수: {stores.type.nunique()}')
print(f'클러스터 개수: {stores.cluster.nunique()}')

도시 개수: 22
국가 개수: 16
타입 개수: 5
클러스터 개수: 17


In [10]:
holidays_events.head()

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False


데이터 설명<br><br>
type: 종류.  Holiday / Event / Additional / Transfer / Bridge / Work Day<br>
locale: 공휴일의 범위. National / Local / Regional<br>
locale_name: 지역 이름.<br>
description: 공휴일 설명.<br>
transferred: 이전 유무.

In [43]:
print(f'공휴일 종류 개수: {holidays_events.type.nunique()}')
print(f'범위 타입 개수: {holidays_events.locale.nunique()}')
print(f'위치 종류 개수: {holidays_events.locale_name.nunique()}')
print(f'이전 종류 개수: {holidays_events.transferred.nunique()}')

공휴일 종류 개수: 6
범위 타입 개수: 3
위치 종류 개수: 24
이전 종류 개수: 2


공공부문 임금은 15일과 이달 말일에 2주 간격으로 지급된다. 슈퍼마켓 매출이 이에 영향을 받을 수 있습니다.<br>
2016년 에콰도르 지진은 2016년 4월 16일 에콰도르에서 발생한 규모 7.8의 지진이다. 사람들은 지진 발생 후 몇 주 동안 슈퍼마켓의 판매에 큰 영향을 미친 물과 다른 일용품들을 기부하는 구호활동에 동참했다.

In [17]:
oil.head()

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20


In [46]:
train = train.merge(holidays_events, on='date', how='left')
train = train.merge(oil, on='date', how='left')
train = train.merge(stores, on='store_nbr', how='left')
train = train.merge(transactions, on=['date', 'store_nbr'], how='left')
train = train.rename(columns= {'type_x': 'holiday_type', 'type_y': 'store_type'})
train['date'] = pd.to_datetime(train['date'])
train['year'] = train['date'].dt.year
train['month'] = train['date'].dt.month
train['week'] = train['date'].dt.isocalendar().week
train['quarter'] = train['date'].dt.quarter
train['day_of_week'] = train['date'].dt.day_name()

In [47]:
train.head()

,id,date,store_nbr,family,sales,onpromotion,holiday_type,locale,locale_name,description,...,city,state,store_type,cluster,transactions,year,month,week,quarter,day_of_week
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0.0,Holiday,National,Ecuador,Primer dia del ano,...,Quito,Pichincha,D,13,NaN,2013,1,1,1,Tuesday
1,1,2013-01-01,1,BABY CARE,0.0,0.0,Holiday,National,Ecuador,Primer dia del ano,...,Quito,Pichincha,D,13,NaN,2013,1,1,1,Tuesday
2,2,2013-01-01,1,BEAUTY,0.0,0.0,Holiday,National,Ecuador,Primer dia del ano,...,Quito,Pichincha,D,13,NaN,2013,1,1,1,Tuesday
3,3,2013-01-01,1,BEVERAGES,0.0,0.0,Holiday,National,Ecuador,Primer dia del ano,...,Quito,Pichincha,D,13,NaN,2013,1,1,1,Tuesday
4,4,2013-01-01,1,BOOKS,0.0,0.0,Holiday,National,Ecuador,Primer dia del ano,...,Quito,Pichincha,D,13,NaN,2013,1,1,1,Tuesday


In [50]:
# data
df_st_sa = train.groupby('store_type').agg({"sales" : "mean"}).reset_index().sort_values(by='sales', ascending=False)
df_fa_sa = train.groupby('family').agg({"sales" : "mean"}).reset_index().sort_values(by='sales', ascending=False)[:10]
df_cl_sa = train.groupby('cluster').agg({"sales" : "mean"}).reset_index() 
# chart color
df_fa_sa['color'] = '#496595'
df_fa_sa['color'][2:] = '#c6ccd8'
df_cl_sa['color'] = '#c6ccd8'

# chart
fig = make_subplots(rows=2, cols=2, 
                    specs=[[{"type": "bar"}, {"type": "pie"}],
                           [{"colspan": 2}, None]],
                    column_widths=[0.7, 0.3], vertical_spacing=0, horizontal_spacing=0.02,
                    subplot_titles=("Top 10 Highest Product Sales", "Highest Sales in Stores", "Clusters Vs Sales"))

fig.add_trace(go.Bar(x=df_fa_sa['sales'], y=df_fa_sa['family'], marker=dict(color= df_fa_sa['color']),
                     name='Family', orientation='h'), 
                     row=1, col=1)
fig.add_trace(go.Pie(values=df_st_sa['sales'], labels=df_st_sa['store_type'], name='Store type',
                     marker=dict(colors=['#334668','#496595','#6D83AA','#91A2BF','#C8D0DF']), hole=0.7,
                     hoverinfo='label+percent+value', textinfo='label'), 
                    row=1, col=2)
fig.add_trace(go.Bar(x=df_cl_sa['cluster'], y=df_cl_sa['sales'], 
                     marker=dict(color= df_cl_sa['color']), name='Cluster'), 
                     row=2, col=1)

# styling
fig.update_yaxes(showgrid=False, ticksuffix=' ', categoryorder='total ascending', row=1, col=1)
fig.update_xaxes(visible=False, row=1, col=1)
fig.update_xaxes(tickmode = 'array', tickvals=df_cl_sa.cluster, ticktext=[i for i in range(1,17)], row=2, col=1)
fig.update_yaxes(visible=False, row=2, col=1)
fig.update_layout(height=500, bargap=0.2,
                  margin=dict(b=0,r=20,l=20), xaxis=dict(tickmode='linear'),
                  title_text="Average Sales Analysis",
                  template="plotly_white",
                  title_font=dict(size=25, color='#8a8d93', family="Lato, sans-serif"),
                  font=dict(color='#8a8d93'),
                  hoverlabel=dict(bgcolor="#f2f2f2", font_size=13, font_family="Lato, sans-serif"),
                  showlegend=False)
fig.show()

In [53]:
# data
df_m_sa = train.groupby('month').agg({"sales" : "mean"}).reset_index()
df_m_sa['sales'] = round(df_m_sa['sales'],2)
df_m_sa['month_text'] = df_m_sa['month'].apply(lambda x: calendar.month_abbr[x])
df_m_sa['text'] = df_m_sa['month_text'] + ' - ' + df_m_sa['sales'].astype(str) 

df_w_sa = train.groupby('week').agg({"sales" : "mean"}).reset_index() 
df_q_sa = train.groupby('quarter').agg({"sales" : "mean"}).reset_index() 
# chart color
df_m_sa['color'] = '#496595'
df_m_sa['color'][:-1] = '#c6ccd8'
df_w_sa['color'] = '#c6ccd8'

# chart
fig = make_subplots(rows=2, cols=2, vertical_spacing=0.08,
                    row_heights=[0.7, 0.3], 
                    specs=[[{"type": "bar"}, {"type": "pie"}],
                           [{"colspan": 2}, None]],
                    column_widths=[0.7, 0.3],
                    subplot_titles=("Month wise Avg Sales Analysis", "Quarter wise Avg Sales Analysis", 
                                    "Week wise Avg Sales Analysis"))

fig.add_trace(go.Bar(x=df_m_sa['sales'], y=df_m_sa['month'], marker=dict(color= df_m_sa['color']),
                     text=df_m_sa['text'],textposition='auto',
                     name='Month', orientation='h'), 
                     row=1, col=1)
fig.add_trace(go.Pie(values=df_q_sa['sales'], labels=df_q_sa['quarter'], name='Quarter',
                     marker=dict(colors=['#334668','#496595','#6D83AA','#91A2BF','#C8D0DF']), hole=0.7,
                     hoverinfo='label+percent+value', textinfo='label+percent'), 
                     row=1, col=2)
fig.add_trace(go.Scatter(x=df_w_sa['week'], y=df_w_sa['sales'], mode='lines+markers', fill='tozeroy', fillcolor='#c6ccd8',
                     marker=dict(color= '#496595'), name='Week'), 
                     row=2, col=1)

# styling
fig.update_yaxes(visible=False, row=1, col=1)
fig.update_xaxes(visible=False, row=1, col=1)
fig.update_xaxes(tickmode = 'array', tickvals=df_w_sa.week, ticktext=[i for i in range(1,53)], 
                 row=2, col=1)
fig.update_yaxes(visible=False, row=2, col=1)
fig.update_layout(height=750, bargap=0.15,
                  margin=dict(b=0,r=20,l=20), 
                  title_text="Average Sales Analysis",
                  template="plotly_white",
                  title_font=dict(size=25, color='#8a8d93', family="Lato, sans-serif"),
                  font=dict(color='#8a8d93'),
                  hoverlabel=dict(bgcolor="#f2f2f2", font_size=13, font_family="Lato, sans-serif"),
                  showlegend=False)
fig.show()

In [54]:
# data
df_dw_sa = train.groupby('day_of_week').agg({"sales" : "mean"}).reset_index()
df_dw_sa.sales = round(df_dw_sa.sales, 2)

# chart
fig = px.bar(df_dw_sa, y='day_of_week', x='sales', title='Avg Sales vs Day of Week',
             color_discrete_sequence=['#c6ccd8'], text='sales',
             category_orders=dict(day_of_week=["Monday","Tuesday","Wednesday","Thursday", "Friday","Saturday","Sunday"]))
fig.update_yaxes(showgrid=False, ticksuffix=' ', showline=False)
fig.update_xaxes(visible=False)
fig.update_layout(margin=dict(t=60, b=0, l=0, r=0), height=350,
                  hovermode="y unified", 
                  yaxis_title=" ", template='plotly_white',
                  title_font=dict(size=25, color='#8a8d93', family="Lato, sans-serif"),
                  font=dict(color='#8a8d93'),
                  hoverlabel=dict(bgcolor="#c6ccd8", font_size=13, font_family="Lato, sans-serif"))